In [2]:
import random
import numpy as np

In [5]:
'''
    Kaspers version with some minor changes to make it more pyton-like
'''
def kasper1(NO_OF_PACKAGES):
    collectionList = []
    #i=0
    #while(i<NO_OF_PACKAGES):
    #    collectionList.append(0)
    #    i+=1

    # Make a list with NO_OF_PACKAGES zeros
    # Idea is to set the value to 1 if we
    # later get this number

    collectionList = [0]*NO_OF_PACKAGES #Un-nice but pytonesqe
    
    #Small change, we open package 0 to NO_OF_PACKAGES-1
    #Since Python uses zero index
    def openPackage():
        philosopher = random.randint(0,NO_OF_PACKAGES - 1) 
        return philosopher

    collected  = 0
    numberOfPackages = 0
    while collected < NO_OF_PACKAGES:
        o = openPackage()
        if collectionList[o] == 0:
            collectionList[o] = 1
            collected +=1
        
        numberOfPackages+=1

    return numberOfPackages


In [7]:
# Idea follows Kaspers quite close, but instead of keeping track of
# the list we make a "bag" with the numbers and pop them out until it's
# empty. I think it's nuce because it's straightforward
def fredrik1(NO_OF_PACKAGES):
    packages = set(range(0,NO_OF_PACKAGES))
    number_of_packages = 0
    while packages:
        packages.discard(random.randint(0,NO_OF_PACKAGES - 1))
        number_of_packages += 1
    
    return number_of_packages

In [9]:
 # Use numpy, otherwise much the same as fredrik1
def fredrik2(NO_OF_PACKAGES):
    packages = np.zeros((NO_OF_PACKAGES,), dtype='bool')
    number_of_packages = 0
    while not packages.all():
        x = np.random.randint(0,NO_OF_PACKAGES)
        packages[x] = True
        number_of_packages +=1
    
    return number_of_packages
    
    

In [10]:
# The idea is to use a bitmap to keep track of the numbers instead
# of the numbers itself (i.e.10 is a NO_OF_PACKAGES bit array with bit 10 set)
# We then do a logical and on the matrix to find row number  of
# first row where all bits are set
# For efficiency, we create a N * NO_OF_PACKAGES matrix and
# set one random bit per row. We then do an accumulative AND operation
# to find first row where all bits are set.
# It is similar to above but it uses bit operations instead of indices
def fredrik3(NO_OF_PACKAGES):   
    N = 100
    foundIt = False
    iter = 0
    while not foundIt:
        packages = np.zeros((N,NO_OF_PACKAGES), dtype='bool')
        # Set bit X in each row to 1 to represent a random number
        x = np.random.randint(0,NO_OF_PACKAGES,(N,)) + np.arange(0,N * NO_OF_PACKAGES,step=NO_OF_PACKAGES)
        packages.flat[x] = True
        # Accumulate each row
        x2 = np.bitwise_or.accumulate(packages,axis=0)
        # Check if all are True
        x3 = np.bitwise_and.reduce(x2,axis=1)
        # Here we use a little trick: argmax returns first occurence
        # of maximum value, i.e True. If we have no Trues it returns zero
        # so we have to make a special check if we have True on first row
        # or if all is false
        k = x3.argmax()
        foundIt = True
        if k > 0:
            foundIt = True
        else:
            if x3[0]:
                foundIt = True
            else:
                iter += 1
        
    
    return N * iter + k + 1
 

In [12]:
# And we want to be able to run lots of time to see the convergence
def openLotsOfPackages(fcn, N):
    # Calculates the average of opening lots of packages.
    # We do this to see where we have a convergence
    NO_OF_PACKAGES = 66
    m = 0
    for i in range(N):
        m +=fcn(NO_OF_PACKAGES)
        
    return m / N

In [16]:
# Now we check the results!
%timeit openLotsOfPackages(kasper1,10000)

4.96 s ± 819 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%timeit openLotsOfPackages(fredrik1,10000)

4.26 s ± 88.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
# Not much gain there, but let's try numpy version
%timeit openLotsOfPackages(fredrik2,10000)

10 s ± 189 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
# Slower??? Why is that? What did I do wrong? Last try, the bitwise matrix version
%timeit openLotsOfPackages(fredrik3,10000)

387 ms ± 43.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
# SUCCESS! It was quite a bit faster